In [ ]:
import torch
from torch import nn
def expect_shape_to_match(x: torch.Tensor, shape):
    assert x.shape[1:
                   ] == shape, f"Expected : {shape} but found: {tuple(x.shape[1:])}"


class HelperModule(nn.Module):
    def __init__(self, expected_shape) -> None:
        super().__init__()
        self.expected_shape = expected_shape

    def custom_forward(self, *args, **kwargs):
        res = self.forward(*args, **kwargs)
        expect_shape_to_match(res[0] if isinstance(
            res, tuple) else res, self.expected_shape)
        return res


class CustomFasterRCNN(nn.Module):
    pass

    def nparams(self):
        return sum([p.numel() for p in self.parameters()])/1e6

model = CustomFasterRCNN()

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn,RetinaNet_ResNet50_FPN_Weights
model = fasterrcnn_resnet50_fpn(pretrained=True)
with open("mscoco_classes.txt") as f: 
    classes = f.read().splitlines()

In [111]:
import matplotlib.pyplot as plt
import cv2
def draw_text(
    img, text,
    pos,
    font=cv2.FONT_HERSHEY_COMPLEX_SMALL,
    font_scale=1,
    font_thickness=1,
    text_color=(255, 255, 255),
    text_color_bg=(0, 0, 0)
):

    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, pos, (x + text_w, y + text_h), text_color_bg, -1)
    cv2.putText(img, text, (x, y + text_h + font_scale - 1), font, font_scale, text_color, font_thickness)
    return text_size
def plot_bbox(image,boxes,labels,scores):
    for box, label, score in zip(boxes.int().tolist(), labels.tolist(), scores.tolist()):
        [x1, y1, x2, y2] = box
        image = cv2.rectangle(image,(x1,y1),(x2,y2),(0,0,0),1)
        classs = classes[label-1]
        draw_text(image,f"{classs}:{score:.1f}",(x1,y1),font_scale=2)
        break
    plt.imshow(image)


image = plt.imread("images/cat.webp")
# image = cv2.resize(image, (image.shape[1]//2, image.shape[0]//2))
# with torch.no_grad():
#     model.eval()
#     res = model.forward([torch.tensor(image).permute(2, 0,1)/255])    
# plot_bbox(image.copy(),res[0]["boxes"], res[0]["labels"], res[0]["scores"])

In [117]:
imgs = torch.tensor(image).permute(2, 0, 1)[None]/255
res = model.backbone(imgs)
res.keys()

odict_keys(['0', '1', '2', '3', 'pool'])